In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time

C:\Users\merel\AppData\Local\Programs\Python\Python313\Lib\site-packages\ema_workbench\em_framework\__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")


In [3]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(2)

In [4]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [5]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [6]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('Expected Annual Damage', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage'), function=<function sum_over at 0x0000020C71EE2340>)
ScalarOutcome('Dike Investment Costs', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs'), function=<function sum_over at 0x0000020C71EE2340>)
ScalarOutcome('RfR Investment Costs', variable_name=('RfR Total Costs',), function=<function sum_over at 0x0000020C71EE2340>)
ScalarOutcome('Evacuation Costs', variable_name=('Expected Evacuation Costs',), function=<function sum_over at 0x0000020C71EE2340>)
ScalarOutcome('Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'), function

In [11]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations

def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy RWS",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1,"2_RfR 0": 1, "EWS_DaysToThreat":3, "A.1_DikeIncrease 0":5, "A.4_DikeIncrease 0":7, "A.5_DikeIncrease 0": 8}
        )
    ),
    Policy(
        "policy RWS 2",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1,"0_RfR 1":1, "2_RfR 0": 1, "2_RfR 1": 1, "EWS_DaysToThreat":3, "A.1_DikeIncrease 0":5, "A.4_DikeIncrease 0":7, "A.5_DikeIncrease 0": 8}
        )
    ),
    Policy(
        "policy RWS 3",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1,"0_RfR 1":1,"0_RfR 2":1, "2_RfR 0": 1, "2_RfR 2":1, "2_RfR 1": 1,  "EWS_DaysToThreat":3, "A.1_DikeIncrease 0":5, "A.4_DikeIncrease 0":7, "A.5_DikeIncrease 0": 8}
        )
    ),
    Policy(
        "policy 7",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 0, "0_RfR 2": 1,
               "2_RfR 0": 1, "2_RfR 1": 1, "2_RfR 2": 0,
               "3_RfR 2": 1,
               "EWS_DaysToThreat":3,
               "A.2_DikeIncrease 0":4,"A.3_DikeIncrease 0":5, "A.4_DikeIncrease 0":3, "A.5_DikeIncrease 0": 2}
        )
    ),

    Policy(
        "policy 9",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "2_RfR 0": 1, "2_RfR 1": 1, "2_RfR 2": 1, "EWS_DaysToThreat":1, "A.2_DikeIncrease 0":3,"A.3_DikeIncrease 0":3, "A.4_DikeIncrease 0":6, "A.5_DikeIncrease 0": 4}
        )
    ),
    Policy(
        "policy 3&2",
        **dict(
            get_do_nothing_dict(),
            **{"3_RfR 0": 1, "3_RfR 1": 1, "3_RfR 2": 1, "2_RfR 0": 1, "2_RfR 1": 1, "2_RfR 2": 1, "EWS_DaysToThreat":1, "A.2_DikeIncrease 0":3,"A.3_DikeIncrease 0":3, "A.4_DikeIncrease 0":6, "A.5_DikeIncrease 0": 4}
        )
    ),
    Policy(
        "policy backup",
        **dict(
            get_do_nothing_dict(),
            **{"2_RfR 0": 1, "2_RfR 1": 1, "2_RfR 2": 1, "3_RfR 0": 1, "3_RfR 1": 0, "3_RfR 2": 0, "EWS_DaysToThreat":0, "A.1_DikeIncrease 0":3,"A.2_DikeIncrease 0":3, "A.3_DikeIncrease 0":4}
        )),
   Policy(
    "policy Overijssel",
    **dict(
        get_do_nothing_dict(),
        **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "2_RfR 0": 1, "2_RfR 1": 1, "2_RfR 2": 1, "EWS_DaysToThreat":1, "A.2_DikeIncrease 0":3,"A.3_DikeIncrease 0":3, "A.4_DikeIncrease 0":5, "A.5_DikeIncrease 0": 4}
    )
) ]

In [ ]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 1000
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 12 workers
[MainProcess/INFO] performing 1000 scenarios * 8 policies * 1 model(s) = 8000 experiments
 15%|█████▋                                | 1202/8000 [02:23<09:31, 11.89it/s]

In [9]:
experiments, outcomes = results
outcomes=pd.DataFrame(outcomes)

In [10]:
merged_df = pd.concat([experiments, outcomes], axis=1)
merged_df.to_csv(r'C:\Users\merel\PycharmProjects\EPA141A\final assignment\Robustness results/robustness_check_RWS_DC_1000_overijssel.csv', index=False)


In [ ]:
Policy(
    "policy Overijssel",
    **dict(
        get_do_nothing_dict(),
        **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "2_RfR 0": 1, "2_RfR 1": 1, "2_RfR 2": 1, "EWS_DaysToThreat":1, "A.2_DikeIncrease 0":3,"A.3_DikeIncrease 0":3, "A.4_DikeIncrease 0":5, "A.5_DikeIncrease 0": 4}
    )
)